# 2003-06-24T04:24:10


Nightlog information for file VIMOS.2003-06-24T04:24:10.847 <br/>
OB grade: C)Out of specs - Repeat <br/>
Comment: Pointing off by 4-5" south (given a 13"x13" field) - **user's source is at edge of field, could still be useful but will be repeated**

                  Requested    Fulfilled?
    Seeing:           1.20          Yes
    Airmass:          1.50          Yes
    Moon Distance:    90          Yes
    Sky Transparency: 3THN          Yes
    Strehl Ratio:     0.000          N/A


In [66]:
import glob
import os
import re
import itertools
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy.io import fits
import numpy as np

def flatten(listOfLists):
    "Flatten one level of nesting"
    return itertools.chain.from_iterable(listOfLists)

In [67]:
def reduceifu(datadirectory, rawdatadictionary):
    """Something"""
    datadir = datadirectory
    rawdata = rawdatadictionary



    # Put to each the data dir and print hte quadran
    for k,v in rawdata.items():
        try:
            rawdata[k] = datadir+v
        except:
            temp = [datadir+i for i in v]
            rawdata[k] = temp

        t = fits.open(rawdata['IFU_SCIENCE'])[0]
        quadrant = t.header['HIERARCH ESO OCS CON QUAD']
    print('Quadrant {}'.format(quadrant))
    
    
    
    ##Create bias
    
    outdir = datadir+'quadrant'+str(quadrant)

    #Create outdir if it doesnt exist
    if not os.path.exists(outdir):
        os.makedirs(outdir)


    filenamebias = outdir+'/bias.sof'
    with open(filenamebias,'w') as file:
        for bias in rawdata['BIAS'][0:]:
            cat = 'BIAS'
            string = '{} {}\n'.format(bias,cat)
            file.write(string)
            
            
                
    stackmethod = '--StackMethod=Average'
    esorexpath = '/bin/esorex'
    routine = 'vmbias'
    outputdir = '--output-dir={}'.format(outdir)
    logdir = '--log-dir={}'.format(outdir)
    
    commandbias = '{} {} {} {}  {}'.format(esorexpath,outputdir,logdir,routine,  filenamebias)
    print(commandbias)
    os.system(commandbias)
    
    os.rename(outdir+'/esorex.log',outdir+'/'+routine+'.log')
    
    rawdata['MASTER_BIAS'] = outdir+'/master_bias.fits'
    
    
    ##Calib
    
    inputfilescalib = ['IFU_SCREEN_FLAT','IFU_ARC_SPECTRUM','LINE_CATALOG','IFU_IDENT','MASTER_BIAS']

    filenamecalib = outdir+'/calib.sof'
    with open(filenamecalib,'w') as file:
        for i in inputfilescalib:
            tmpfile = rawdata[i]
            if isinstance(tmpfile, list):    
                for j in tmpfile:
                    string = '{} {}\n'.format(j,i)
                    file.write(string)
            else:
                string = '{} {}\n'.format(tmpfile,i)
                file.write(string)


    # Define the routine and call it
    routine = 'vmifucalib'

    commandcalib = '{} {} {} {}  {}'.format(esorexpath,outputdir,logdir,routine,  filenamecalib)
    print(commandcalib)
    os.system(commandcalib)
    
    os.rename(outdir+'/esorex.log',outdir+'/'+routine+'.log')

    rawdata['IFU_IDS'] = outdir+'/ifu_ids.fits'
    rawdata['IFU_TRACE'] = outdir+'/ifu_trace.fits'
    rawdata['IFU_TRANSMISSION'] = outdir+'/ifu_transmission.fits'
    
    
    ##Standard
    inputfilesstandard = ['IFU_STANDARD','MASTER_BIAS','IFU_IDS','IFU_TRACE','IFU_TRANSMISSION','EXTINCT_TABLE','STD_FLUX_TABLE']

    filenamestandard = outdir+'/ifustandard.sof'
    with open(filenamestandard,'w') as file:
        for i in inputfilesstandard:
            tmpfile = rawdata[i]
            if isinstance(tmpfile, list):    
                for j in tmpfile:
                    string = '{} {}\n'.format(j,i)
                    file.write(string)
            else:
                string = '{} {}\n'.format(tmpfile,i)
                file.write(string)

                routine = 'vmifustandard'

    commandstandard = '{} {} {} {}  {}'.format(esorexpath,outputdir,logdir,routine,  filenamestandard)
    print(commandstandard)
    os.system(commandstandard)
    os.rename(outdir+'/esorex.log',outdir+'/'+routine+'.log')

    #Science Soif
    
    rawdata['IFU_SPECPHOT_TABLE'] = outdir+'/ifu_specphot_table.fits'

    inputfilesscience = ['IFU_SCIENCE','MASTER_BIAS','IFU_IDS','IFU_TRACE','IFU_TRANSMISSION','EXTINCT_TABLE','IFU_SPECPHOT_TABLE']

    filenamescience = outdir+'/ifuscience.sof'
    with open(filenamescience,'w') as file:
        for i in inputfilesscience:
            tmpfile = rawdata[i]
            if isinstance(tmpfile, list):    
                for j in tmpfile:
                    string = '{} {}\n'.format(j,i)
                    file.write(string)
            else:
                string = '{} {}\n'.format(tmpfile,i)
                file.write(string)
                
                
                routine = 'vmifuscience'

    calibrateflux = '--CalibrateFlux=true'

    commandscience = '{} {} {} {} {} {}'.format(esorexpath,outputdir,logdir,routine, calibrateflux ,filenamescience)
    print(commandscience)
    os.system(commandscience)
    os.rename(outdir+'/esorex.log',outdir+'/'+routine+'.log')


inputs

In [68]:
datadir = '/home/mmarcano/Documents/VIMOS/NGC6652/data/NGC6652B4/2003-06-24T04:24:10/RawESOdata/'

rawdataq1 = {'IFU_SCIENCE':'VIMOS.2003-06-24T04:24:10.847.fits',
           'LINE_CATALOG':'M.VIMOS.2008-06-17T18:11:18.672.fits',
          'EXTINCT_TABLE':'M.VIMOS.2008-06-17T18:12:53.176.fits',
          'STD_FLUX_TABLE':'M.VIMOS.2008-06-17T18:13:13.956.fits',
           'IFU_IDENT':'M.VIMOS.2008-06-17T18:16:03.380.fits',
           'IFU_MASTER_RESPONSE':'M.VIMOS.2008-06-17T18:20:40.981.fits',
           'BIAS':['VIMOS.2003-06-23T14:48:34.459.fits','VIMOS.2003-06-23T14:49:26.872.fits','VIMOS.2003-06-23T14:50:19.472.fits','VIMOS.2003-06-23T14:51:12.021.fits','VIMOS.2003-06-23T14:52:04.693.fits',
                  'VIMOS.2003-06-24T11:27:11.124.fits','VIMOS.2003-06-24T11:28:03.627.fits','VIMOS.2003-06-24T11:28:56.486.fits','VIMOS.2003-06-24T11:29:49.973.fits','VIMOS.2003-06-24T11:30:43.186.fits'],
           'IFU_ARC_SPECTRUM':['VIMOS.2003-06-24T05:41:51.285.fits'],
           'IFU_STANDARD':'VIMOS.2003-06-24T10:26:46.402.fits',
           'IFU_SCREEN_FLAT':['VIMOS.2003-06-24T10:43:31.636.fits','VIMOS.2003-06-24T10:44:42.776.fits','VIMOS.2003-06-24T10:45:54.168.fits']}


rawdataq4 = {'IFU_SCIENCE':'VIMOS.2003-06-24T04:24:10.848.fits',
           'LINE_CATALOG':'M.VIMOS.2008-06-17T18:11:31.782.fits',
          'EXTINCT_TABLE':'M.VIMOS.2008-06-17T18:12:53.176.fits',
          'STD_FLUX_TABLE':'M.VIMOS.2008-06-17T18:13:13.956.fits',
           'IFU_IDENT':'M.VIMOS.2008-06-17T18:16:12.090.fits',
           'IFU_MASTER_RESPONSE':'M.VIMOS.2008-06-17T18:20:50.827.fits',
           'BIAS':['VIMOS.2003-06-23T14:48:34.460.fits','VIMOS.2003-06-23T14:49:26.873.fits','VIMOS.2003-06-23T14:50:19.473.fits','VIMOS.2003-06-23T14:51:12.022.fits','VIMOS.2003-06-23T14:52:04.694.fits',
                  'VIMOS.2003-06-24T11:27:11.125.fits','VIMOS.2003-06-24T11:28:03.628.fits','VIMOS.2003-06-24T11:28:56.487.fits','VIMOS.2003-06-24T11:29:49.974.fits','VIMOS.2003-06-24T11:30:43.187.fits'],
           'IFU_ARC_SPECTRUM':'VIMOS.2003-06-24T05:41:51.286.fits',
           'IFU_STANDARD':'VIMOS.2003-06-24T10:38:01.522.fits',
           'IFU_SCREEN_FLAT':['VIMOS.2003-06-24T10:43:31.637.fits','VIMOS.2003-06-24T10:44:42.777.fits','VIMOS.2003-06-24T10:45:54.169.fits']
          }


rawdataq2 = {'IFU_SCIENCE':'VIMOS.2003-06-24T04:24:10.986.fits',
           'LINE_CATALOG':'M.VIMOS.2008-06-17T18:11:23.749.fits',
          'EXTINCT_TABLE':'M.VIMOS.2008-06-17T18:12:53.176.fits',
          'STD_FLUX_TABLE':'M.VIMOS.2008-06-17T18:13:13.956.fits',
           'IFU_IDENT':'M.VIMOS.2008-06-17T18:16:06.327.fits',
           'IFU_MASTER_RESPONSE':'M.VIMOS.2008-06-17T18:20:44.067.fits',
           'BIAS':['VIMOS.2003-06-23T14:48:34.615.fits','VIMOS.2003-06-23T14:49:27.030.fits','VIMOS.2003-06-23T14:50:19.637.fits','VIMOS.2003-06-23T14:51:12.165.fits','VIMOS.2003-06-23T14:52:04.864.fits',
                  'VIMOS.2003-06-24T11:27:11.267.fits','VIMOS.2003-06-24T11:28:03.762.fits','VIMOS.2003-06-24T11:28:56.625.fits','VIMOS.2003-06-24T11:29:50.113.fits','VIMOS.2003-06-24T11:30:43.334.fits'],
           'IFU_ARC_SPECTRUM':'VIMOS.2003-06-24T05:41:51.424.fits',
           'IFU_STANDARD':'VIMOS.2003-06-24T10:30:31.616.fits',
           'IFU_SCREEN_FLAT':['VIMOS.2003-06-24T10:43:31.777.fits','VIMOS.2003-06-24T10:44:42.923.fits','VIMOS.2003-06-24T10:45:54.300.fits']
          }


rawdataq3 = {'IFU_SCIENCE':'VIMOS.2003-06-24T04:24:10.987.fits',
           'LINE_CATALOG':'M.VIMOS.2008-06-17T18:11:28.741.fits',
          'EXTINCT_TABLE':'M.VIMOS.2008-06-17T18:12:53.176.fits',
          'STD_FLUX_TABLE':'M.VIMOS.2008-06-17T18:13:13.956.fits',
           'IFU_IDENT':'M.VIMOS.2008-06-17T18:16:09.125.fits',
           'IFU_MASTER_RESPONSE':'M.VIMOS.2008-06-17T18:20:47.901.fits',
           'BIAS':['VIMOS.2003-06-23T14:48:34.616.fits','VIMOS.2003-06-23T14:49:27.031.fits','VIMOS.2003-06-23T14:50:19.638.fits','VIMOS.2003-06-23T14:51:12.166.fits','VIMOS.2003-06-23T14:52:04.865.fits',
                  'VIMOS.2003-06-24T11:27:11.268.fits','VIMOS.2003-06-24T11:28:03.763.fits','VIMOS.2003-06-24T11:28:56.626.fits','VIMOS.2003-06-24T11:29:50.114.fits','VIMOS.2003-06-24T11:30:43.335.fits'],
           'IFU_ARC_SPECTRUM':'VIMOS.2003-06-24T05:41:51.425.fits',
           'IFU_STANDARD':'VIMOS.2003-06-24T10:34:16.581.fits',
           'IFU_SCREEN_FLAT':['VIMOS.2003-06-24T10:43:31.778.fits','VIMOS.2003-06-24T10:44:42.924.fits','VIMOS.2003-06-24T10:45:54.301.fits']
          }

In [69]:
reduceifu(datadir,rawdataq1)
reduceifu(datadir,rawdataq4)
reduceifu(datadir,rawdataq2)
reduceifu(datadir,rawdataq3)




Quadrant 1
/bin/esorex --output-dir=/home/mmarcano/Documents/VIMOS/NGC6652/data/NGC6652B4/2003-06-24T04:24:10/RawESOdata/quadrant1 --log-dir=/home/mmarcano/Documents/VIMOS/NGC6652/data/NGC6652B4/2003-06-24T04:24:10/RawESOdata/quadrant1 vmbias  /home/mmarcano/Documents/VIMOS/NGC6652/data/NGC6652B4/2003-06-24T04:24:10/RawESOdata/quadrant1/bias.sof
/bin/esorex --output-dir=/home/mmarcano/Documents/VIMOS/NGC6652/data/NGC6652B4/2003-06-24T04:24:10/RawESOdata/quadrant1 --log-dir=/home/mmarcano/Documents/VIMOS/NGC6652/data/NGC6652B4/2003-06-24T04:24:10/RawESOdata/quadrant1 vmifucalib  /home/mmarcano/Documents/VIMOS/NGC6652/data/NGC6652B4/2003-06-24T04:24:10/RawESOdata/quadrant1/calib.sof
/bin/esorex --output-dir=/home/mmarcano/Documents/VIMOS/NGC6652/data/NGC6652B4/2003-06-24T04:24:10/RawESOdata/quadrant1 --log-dir=/home/mmarcano/Documents/VIMOS/NGC6652/data/NGC6652B4/2003-06-24T04:24:10/RawESOdata/quadrant1 vmifustandard  /home/mmarcano/Documents/VIMOS/NGC6652/data/NGC6652B4/2003-06-24T04:24

In [70]:
combinedir = datadir+'Combine'

#Create outdir if it doesnt exist
if not os.path.exists(combinedir):
    os.makedirs(combinedir)



filenamescombinecube = combinedir+'/ifucombinefov.sof'

with open(filenamescombinecube,'w') as file:
    for i in [1,2,3,4]:
        fitstemp='{}quadrant{}/ifu_science_flux_reduced.fits'.format(datadir,i)
        string = '{} {}\n'.format(fitstemp,'IFU_SCIENCE_FLUX_REDUCED')
        file.write(string)
        
esorexpath = '/bin/esorex'
outputdir = '--output-dir={}'.format(combinedir)
logdir = '--log-dir={}'.format(combinedir)
routine = 'vmifucombinecube'
command = '{} {} {} {} {} '.format(esorexpath,outputdir,logdir,routine, filenamescombinecube)
print(command)
os.system(command)

/bin/esorex --output-dir=/home/mmarcano/Documents/VIMOS/NGC6652/data/NGC6652B4/2003-06-24T04:24:10/RawESOdata/Combine --log-dir=/home/mmarcano/Documents/VIMOS/NGC6652/data/NGC6652B4/2003-06-24T04:24:10/RawESOdata/Combine vmifucombinecube /home/mmarcano/Documents/VIMOS/NGC6652/data/NGC6652B4/2003-06-24T04:24:10/RawESOdata/Combine/ifucombinefov.sof 


0